In [1]:
import os
os.chdir("/dust3r")
print(os.getcwd())

import numpy as np
import torch
from masked_dust3r.scripts.utils.math import *

device = "cpu"

/dust3r


In [2]:
p0 = np.eye(4)
p1 = np.eye(4)
p1[:2,:2] = np.array([[0,1],[-1,0]])

p1[0,3] = 1
p1[1,3] = -1

step = 4

interpolate_pose(p0, p1, step, device=device)


tensor([[[ 1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 9.2388e-01,  3.8268e-01,  0.0000e+00,  2.5000e-01],
         [-3.8268e-01,  9.2388e-01, -0.0000e+00, -2.5000e-01],
         [-0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 7.0711e-01,  7.0711e-01,  0.0000e+00,  5.0000e-01],
         [-7.0711e-01,  7.0711e-01, -0.0000e+00, -5.0000e-01],
         [-0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 3.8268e-01,  9.2388e-01,  0.0000e+00,  7.5000e-01],
         [-9.2388e-01,  3.8268e-01, -0.0000e+00, -7.5000e-01],
         [-0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00, 